# Web and GUI Testing

We apply our techniques on Graphical User Interfaces (GUIs), notably on Web interfaces.

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

## A Web User Interface

In [ ]:
import fuzzingbook_utils

In [ ]:
from IPython.core.display import HTML, display

In [ ]:
fuzzingbook_swag = {
    "tshirt": "One FuzzingBook T-Shirt",
    "drill": "One FuzzingBook Rotary Hammer",
    "lockset": "One FuzzingBook Lock Set"
}

In [ ]:
html_order_form = """
<form action="/order" style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong style="font-size: x-large">Fuzzingbook Swag Order Form</strong>
  <p>
  Yes! Please send me at your earliest convenience
  <select name="item">
  """

for item in fuzzingbook_swag:
    html_order_form += '<option value="{item}">{name}</option>'.format(item=item, 
        name=fuzzingbook_swag[item])

html_order_form += """
  </select>
  <br>
  <table>
  <tr><td>
  <label for="name">Name: </label><input type="text" name="name">
  </td><td>
  <label for="email">Email: </label><input type="email" name="email"><br>
  </td></tr>
  <tr><td>
  <label for="city">City: </label><input type="text" name="city">
  </td><td>
  <label for="zip">ZIP Code: </label><input type="number" name="zip">
  </tr></tr>
  </table>
  <input type="checkbox" name="tandc"><label for="tandc">I have read 
  the <a href="/">terms and conditions</a></label><br>
  <button>Place order</button>
</p>
</form>
"""

In [ ]:
HTML(html_order_form)

In [ ]:
html_order_received = """
<div style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <strong style="font-size: x-large">Thank you for your Fuzzingbook Order!</strong>
  <p>
  We will send <strong>{item_name}</strong> to {name} in {city}, {zip}<br>
  A confirmation mail will be sent to {email}.
  </p>
</div>
"""

In [ ]:
HTML(html_order_received.format(item_name="One FuzzingBook Rotary Hammer", 
                                name="Jane Doe", 
                                email="doe@example.com",
                                city="Seattle",
                                zip="98104"))

1. Start a Web Server (in the background)
2. Send it a request, say "GET /"
3. Get the Web Form
4. Create a "GET" string

To render, check out imgkit: https://pypi.org/project/imgkit/

In [ ]:
from multiprocessing import Process

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler, HTTPStatus

In [ ]:
import urllib.parse

In [ ]:
import html

In [ ]:
class MyHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_HEAD(self):
        print("HEAD " + self.path)
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "text/html")
        self.end_headers()
            
    def do_GET(self):
        print("GET " + self.path)
        if self.path == "/":
            self.send_order_form()
        elif self.path.startswith("/order"):
            self.send_order_received()
        else:
            self.send_response(HTTPStatus.NOT_FOUND, "Not found")

In [ ]:
class MyHTTPRequestHandler(MyHTTPRequestHandler):
    def send_order_form(self):
        self.send_response(HTTPStatus.OK, "Place your order")
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(html_order_form.encode("utf8"))

In [ ]:
from Coverage import cgi_decode

In [ ]:
class MyHTTPRequestHandler(MyHTTPRequestHandler):
    def get_field_values(self):
        # self.path is sth like "/order?item=foo&name=bar"
        # Note: this fails to decode non-ASCII characters properly
        query_string = urllib.parse.urlparse(self.path).query
        
        # fields is { 'item': ['tshirt'], 'name': ['Jane Doe'], ...}
        fields = urllib.parse.parse_qs(query_string, keep_blank_values=True)

        values = {}
        html_values = {}
        for key in fields:
            values[key] = urllib.parse.unquote(html.unescape(fields[key][0]))
            html_values[key] = html.escape(urllib.parse.unquote(values[key]))
            
        return values, html_values

In [ ]:
class MyHTTPRequestHandler(MyHTTPRequestHandler):
    def send_order_received(self):
        try:
            values, html_values = self.get_field_values()
            values["item_name"] = fuzzingbook_swag[values["item"]]
            html_values["item_name"] = html.escape(values["item_name"])

            confirmation = html_order_received.format(**html_values).encode("utf8")

            self.send_response(HTTPStatus.OK, "Order received")
            self.send_header("Content-type", "text/html")
            self.end_headers()
            self.wfile.write(confirmation)
        except Exception:
            self.internal_server_error()

In [ ]:
class MyHTTPRequestHandler(MyHTTPRequestHandler):
    def not_found(self):
        self.send_response(HTTPStatus.NOT_FOUND, "Not found")

In [ ]:
html_internal_server_error = """
<form action="/order" style="border:3px; border-style:solid; border-color:#FF0000; padding: 1em;">
  <h3>Internal Server Error</h3>
  The server has encountered an internal error.  Please try again later.
  <pre>{error_message}</pre>
  """

In [ ]:
import sys
import traceback

In [ ]:
class MyHTTPRequestHandler(MyHTTPRequestHandler):
    def internal_server_error(self):
        self.send_response(HTTPStatus.INTERNAL_SERVER_ERROR, "Internal Error")
        self.send_header("Content-type", "text/html")
        self.end_headers()

        exc = traceback.format_exc()
        print(exc, file=sys.stderr)

        message = html_internal_server_error.format(error_message=exc)
        self.wfile.write(message.encode("utf8"))

In [ ]:
def run_httpd():
    host = "127.0.0.1"
    for port in range(8800, 9000):
        httpd_address = (host, port)
        # httpd_url = "http://" + httpd_address[0] + ":" + repr(httpd_address[1]) + "/"
        
        try:
            httpd = HTTPServer(httpd_address, MyHTTPRequestHandler)
            break
        except OSError:
            continue

    httpd_url = "http://" + host + ":" + repr(port) + "/"
    print("Running at", httpd_url)
    with open("httpd_url.txt", "w") as f:
        f.write(httpd_url)
    httpd.serve_forever()    

In [ ]:
http_process = Process(target=run_httpd)
http_process.start()

In [ ]:
import time

In [ ]:
time.sleep(1)
with open("httpd_url.txt") as f:
    httpd_url = f.read()
httpd_url

In [ ]:
HTML('<a href="' + httpd_url + '">' + httpd_url + "</a>")

In [ ]:
from Carver import webbrowser

In [ ]:
contents = webbrowser(httpd_url)

In [ ]:
HTML(contents)

How can we test this?  By sending one command after another.

In [ ]:
HTML(webbrowser(httpd_url + 
                "order?item=tshirt&name=Jane+Doe&email=doe%40example.com&city=Seattle&zip=98104"))

## Fuzzing A Web Form

### Fuzzing with Expected Values

In [ ]:
from Grammars import crange, is_valid_grammar

In [ ]:
ORDER_GRAMMAR = {
    "<start>": [ "<order>" ],
    "<order>": [ "order?item=<item>&name=<name>&email=<email>&city=<city>&zip=<zip>" ],
    "<item>": [ "tshirt", "drill", "lockset" ],
    "<name>": [ "Jane+Doe", "John+Smith" ],
    "<email>": [ "foo%40example.com"],
    "<city>": [ "Seattle", "New+York"],
    "<zip>": [ "<digit>" * 5 ],
    "<digit>": crange('0', '9')
}
assert is_valid_grammar(ORDER_GRAMMAR)

In [ ]:
from GrammarFuzzer import GrammarFuzzer

In [ ]:
order_fuzzer = GrammarFuzzer(ORDER_GRAMMAR)
[order_fuzzer.fuzz() for i in range(5)]

In [ ]:
url = httpd_url + order_fuzzer.fuzz()
HTML(webbrowser(url))

### Fuzzing with Unexpected Values

So far, so good.  But what happens when we enter random values?

In [ ]:
from MutationFuzzer import MutationFuzzer

In [ ]:
seed = order_fuzzer.fuzz()
seed

In [ ]:
mutate_order_fuzzer = MutationFuzzer([seed], min_mutations=1, max_mutations=1)
[mutate_order_fuzzer.fuzz() for i in range(5)]

In [ ]:
from ExpectError import ExpectError

In [ ]:
import traceback

In [ ]:
import urllib

In [ ]:
while True:
    url = httpd_url + mutate_order_fuzzer.fuzz()
    try:
        answer = webbrowser(url)
    except urllib.request.HTTPError:
        print(url)
        traceback.print_exc()
        break

In [ ]:
from Reducer import DeltaDebuggingReducer

In [ ]:
from Fuzzer import Runner

In [ ]:
class WebRunner(Runner):
    def run(self, inp):
        url = httpd_url + inp
        try:
            answer = webbrowser(url)
            return inp, Runner.PASS
        except urllib.request.HTTPError:
            return inp, Runner.FAIL
        except Exception:
            return inp, Runner.UNRESOLVED

In [ ]:
web_runner = WebRunner()

In [ ]:
while True:
    failing_input, outcome = web_runner.run(mutate_order_fuzzer.fuzz())
    if outcome == Runner.FAIL:
        break

In [ ]:
failing_input

In [ ]:
web_reducer = DeltaDebuggingReducer(web_runner)

In [ ]:
web_reducer.reduce(failing_input)

In [ ]:
with ExpectError():
    webbrowser(httpd_url + "order")

We see that we have a lot to do to make our Web Server more robust against unexpected inputs.

### Crafting Attacks

More interesting, though: Values that are not as common as these

In [ ]:
BAD_ORDER_GRAMMAR = {
    "<name>": [ "Robert'; drop table students; --"],  # https://xkcd.com/327/
    "<city>": [ "Mötley Crüe" ],
}
...

## Extracting Grammars for Web GUIs

Continue with:

* Extract generic grammar from HTML automatically (including bad inputs)

Then, add selenium to fuzz arbitrary Web pages

* Adding selenium https://selenium-python.readthedocs.io
* Synthesize selenium calls


Clean up:

In [ ]:
time.sleep(5)
http_process.terminate()

In [ ]:
import os

In [ ]:
os.remove("httpd_url.txt")

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_